Based on an example by Sebastian Raschka (sebastianraschka.com)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from matplotlib.patches import FancyArrowPatch

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [ ]:
data = np.array([[0.1, 0.2, 0.3], [0.5, 0.6, 0.7], [0.3, 0.4, 0.55], [0.4, 0.47, 0.58], [0.6, 0.72, 0.79], [0.5, 0.45, 0.4], [0.3, 0.7, 0.2], [0.9, 0.4, 0.5]])

n = data.shape[0]

In [ ]:
np.random.seed(1)

n_samples = 40

a_gen = 2
b_gen = 1
c_gen = 2
d_gen = 1

sigma = np.sqrt(0.05)

X1 = np.random.uniform(size=n_samples)
X2 = np.random.uniform(size=n_samples)

X3 = (d_gen - a_gen * X1 - b_gen * X2) / c_gen + np.random.normal(0, sigma, size=n_samples)

data = np.concatenate([[X1], [X2], [X3]], ).T
n = data.shape[0]

In [ ]:
# Plot data points
%matplotlib notebook

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

for i in range(n):
    ax.scatter(data[i, 0], data[i, 1], data[i, 2])
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('X3')

plt.show()

In [ ]:
# Project data to coordinate axes
fig = plt.figure()

ax1 = fig.add_subplot(131)
#ax1.set_xlim(0, 1)

for i in range(n):
    ax1.scatter(data[i, 0], [0])
    
ax1.set_title('Projection to X1')
    
var1 = np.var(data[:, 0])
print('Variance w.r.t X1: %s' % var1)

ax1 = fig.add_subplot(132)
#ax1.set_xlim(0, 1)
ax1.get_yaxis().set_visible(False)

for i in range(n):
    ax1.scatter(data[i, 1], [0])
    
ax1.set_title('Projection to X2')

var2 = np.var(data[:, 1])
print('Variance w.r.t X2: %s' % var2)

ax1 = fig.add_subplot(133)
#ax1.set_xlim(0, 1)
ax1.get_yaxis().set_visible(False)

for i in range(n):
    ax1.scatter(data[i, 2], [0])
    
ax1.set_title('Projection to X3')

var3 = np.var(data[:, 2])
print('Variance w.r.t X3: %s' % var3)

plt.show()

## PCA

In [ ]:
# Compute means of each feature
mean_x = np.mean(data[:,0])
mean_y = np.mean(data[:,1])
mean_z = np.mean(data[:,2])

mean_vector = np.array([[mean_x],[mean_y],[mean_z]])

print('Mean Vector:', mean_vector)

In [ ]:
# Center the data
X = np.matrix(data)
X[:,0] -= mean_x
X[:,1] -= mean_y
X[:,2] -= mean_z

print('Mean after centering: %s' % np.mean(X))

In [ ]:
# Compute covariance matrix
n = X.shape[0]
S = 1.0/n*X.T*X

print(S)

In [ ]:
# Eigenvectors and eigenvalues  from the covariance matrix
eig_val, eig_vec = np.linalg.eig(S)

for i in range(len(eig_val)):
    eigvec = eig_vec[:,i].reshape(1,3).T

    print('Eigenvector {}: \n{}'.format(i+1, eigvec))
    print('Eigenvalue {}: {}'.format(i+1, eig_val[i]))
    print(40 * '-')


In [ ]:
# Print products of eigenvectors to check that they are orthogonal
print(float(eig_vec[0]*eig_vec[1].T))
print(float(eig_vec[0]*eig_vec[2].T))
print(float(eig_vec[1]*eig_vec[2].T))

In [ ]:
# Print norms of eigenvectors
print(np.linalg.norm(eig_vec[0]))
print(np.linalg.norm(eig_vec[1]))
print(np.linalg.norm(eig_vec[2]))

In [ ]:
# Check that the sum of the eigenvalues equals to the sum of the variances
print('Sum of the eigenvalues: %s' % np.sum(eig_val))
print('Sum of the variances: %s' % np.sum(np.diag(S)))

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_val[i]), eig_vec[:,i]) for i in range(len(eig_val))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
comb_var = list()
var = 0
for i in eig_pairs:
    print(i[0])
    var += i[0]
    comb_var.append(var)

In [ ]:
# Plot data points with eigenvectors
%matplotlib notebook

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

for i in range(n):
    ax.scatter(data[i, 0], data[i, 1], data[i, 2])
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('X3')

scaling_factor = 3

for v in eig_vec.T:
    a = Arrow3D([mean_x, mean_x + v[0,0]/scaling_factor], [mean_y, mean_y + v[0,1]/scaling_factor], [mean_z, mean_z + v[0,2]/scaling_factor], mutation_scale=20, lw=3, arrowstyle="-|>", color="r")
    ax.add_artist(a)

plt.show()

In [ ]:
# Create a projection matrix with the first and second principal component
nr_of_pc = 2 # Specify the number of principal components

if nr_of_pc == 1:
    matrix_w = eig_pairs[0][1].reshape(3,1)
elif nr_of_pc == 2:
    matrix_w = np.hstack((eig_pairs[0][1].reshape(3,1), eig_pairs[1][1].reshape(3,1)))
else:
    matrix_w = np.hstack((eig_pairs[0][1].reshape(3,1), eig_pairs[1][1].reshape(3,1), eig_pairs[2][1].reshape(3,1)))

print('Matrix W:', matrix_w)

p = matrix_w.shape[1]
exp_var = comb_var[p-1]/np.sum(eig_val)
print('Explained variance: %s' % exp_var)

In [ ]:
# Transform data to a lower dimensional space
transformed = X*matrix_w
print(np.cov(transformed.T, bias=True))

In [ ]:
# Plot the low dimensional representation

plt.figure()

for i in range(n):
    if nr_of_pc > 1:
        plt.scatter(transformed[i, 0], transformed[i, 1])
    else:
        plt.scatter(transformed[i, 0], 0) 
        
plt.xlabel('PC1')
if nr_of_pc > 1:
    plt.ylabel('PC2')
plt.show()

In [ ]:
# Transform points back to the original space
re_transformed = transformed*matrix_w.T + np.vstack(mean_vector.T)
mean_x1 = np.mean(re_transformed[:,0])
mean_y1 = np.mean(re_transformed[:,1])
mean_z1 = np.mean(re_transformed[:,2])

In [ ]:
# Plot data points after transformation back to original space

fig = plt.figure(figsize=(8,8))
ax2 = fig.add_subplot(111, projection='3d')

for i in range(n):
    ax2.scatter(re_transformed[i, 0], re_transformed[i, 1], re_transformed[i, 2])
ax2.set_xlabel('X1')
ax2.set_ylabel('X2')
ax2.set_zlabel('X3')

plt.show()

In [ ]:
# Formula for a line (if we use only the first principal component)
if nr_of_pc == 1:
    v0 = np.array(matrix_w[:, 0]).reshape(-1,)
    p0 = re_transformed[0, :]

    points = []

    for t in [-1.1, 0.5]:
        points.append(np.array(p0 + v0*t)[0])

In [ ]:
# Formula for plane (if we use the firs two principal components)
if nr_of_pc == 2:
    v1 = np.array(matrix_w[:, 0]).reshape(-1,)
    v2 = np.array(matrix_w[:, 1]).reshape(-1,)

    cp = np.cross(v1, v2)
    a, b, c = cp

    d = np.dot(cp, re_transformed[0,:].reshape(-1, 1))

    xx = np.linspace(-0.1, 1.1, 5)
    yy = np.linspace(-0.1, 1.1, 5)
    XX, YY = np.meshgrid(xx, yy)

    ZZ = (d - a * XX - b * YY) / c


In [ ]:
# Project points back to the original space

%matplotlib notebook

show_eigenvectors = False

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10   

if nr_of_pc == 1:
    ax.plot([points[0][0], points[1][0]], [points[0][1], points[1][1]], [points[0][2], points[1][2]], c='magenta') # Line for 1-dimensional model
elif nr_of_pc == 2:
    surf = ax.plot_surface(XX, YY, ZZ, antialiased=False, color='magenta', alpha=0.1) # Plane for 2-dimensional model

for i in range(n):
    ax.scatter(data[i,0], data[i,1], data[i,2], 'o', s=30, c='blue', label='original points' )
    ax.scatter(re_transformed[i,0], re_transformed[i,1], re_transformed[i,2], '*', s=30, c='magenta', label='projected points' )
    plt.plot([re_transformed[i,0], data[i,0]], [re_transformed[i,1], data[i,1]],[re_transformed[i,2], data[i,2]], c='red')

if show_eigenvectors:
    for v in eig_vec.T:
        a = Arrow3D([mean_x, mean_x + v[0,0]], [mean_y, mean_y + v[0,1]], [mean_z, mean_z + v[0,2]], mutation_scale=20, lw=3, arrowstyle="-|>", color="r")
        ax.add_artist(a)
    
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('X3')

plt.show()
